# Red Hat Business Value

We will use the dataset provided for the “Red Hat Business Value” competition. This competition was hosted on Kaggle a few years back, and the dataset is a really good business use case for our study. The archived competition is available at www.kaggle.com/c/predicting-red-hat-business-value.

## 1) Problem Statement

It highlights the problem that deals with predicting high-value customers for their business based on the operational interaction data and thereby helping the company effectively prioritize resources to generate more business and serve its customers better.

* The organization is an American multinational software company that provides open source software products to the enterprise community. 
* Their primary product is Red Hat Enterprise Linux, the most popular distribution of Linux OS, used by various large enterprises. 
* In its services, it helps organizations align their IT strategies by providing enterprise-grade solutions through an open business model and an affordable, predictable subscription model. 
* These subscriptions from large enterprise customers create a substantial part of their revenue, and therefore it is of paramount importance for them to understand their valuable customers and serve them better by prioritizing resources and strategies to drive improved business value.

Red Hat has been in existence for over 25 years. In the long stint of business, they have accumulated and captured a vast amount of data from customer interactions and their descriptive attributes. This rich source of data could be a gold mine of patterns that can help in identifying a potential customer by studying the vast and complex historical patterns in the interaction data.

With the ever-growing popularity and prowess of DL, we can develop a DNN that can learn from historic customer attributes and operational interaction data to understand the deep patterns and predict whether a new customer will potentially be a high-value customer for various business services.

## 2) Exploring the Data

In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [191]:
#Import the 2 datasets provided in the Zip Folder
activity_df = pd.read_csv("act_train.csv")
people_df = pd.read_csv("people.csv")

In [192]:
#Explore the shape of the datasets
print("Shape of Activities DF:",activity_df.shape)
print("Shape of People DF:",people_df.shape)

Shape of Activities DF: (2197291, 15)
Shape of People DF: (189118, 41)


In [194]:
#Explore the contents of the first dataset
activity_df.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


Exploring the contents of the training dataset, we can see that it mostly has customer interaction data but is completely anonymized.

In [195]:
#Calculating the % of Null values in each column for activity data
activity_df.isnull().sum()/activity_df.shape[0]

people_id            0.000000
activity_id          0.000000
date                 0.000000
activity_category    0.000000
char_1               0.928268
char_2               0.928268
char_3               0.928268
char_4               0.928268
char_5               0.928268
char_6               0.928268
char_7               0.928268
char_8               0.928268
char_9               0.928268
char_10              0.071732
outcome              0.000000
dtype: float64

#### Around nine features have more than 90% null values. We can’t do much to fix these features. [char_1 to char_9]

In [196]:
#missing values/null values
activity_df.isnull().sum().sum()

18514699

In [197]:
#Explore the contents of People dataset
people_df.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [198]:
#Calculate the % of null values in for the entire dataset
people_df.isnull().sum().sum()

0

#### And we see that none of the columns in the customer dataset has missing values. But before we do that, we need to take care of a few things as below:

* To create a consolidated dataset, we need to join the activity and customer data on the people_id key. 
* We need to drop the columns in the activity data that have 90% missing values, as they cannot be fixed. [char_1 to char_9]
* Secondly, the “date” and “char_10” columns are present in both datasets. 
* In order to avoid a nameclash, let us rename the “date” column in the activity dataset to “activity_date” and “char_10” in the activity data as “activity_type.” 
* Next, we also need to fix the missing values in the “activity_type” column. 
* Once these two tasks are accomplished, we will join the two datasets and explore the consolidated data.

In [199]:
#Create the list of columns to drop from activity data
columns_to_remove = ["char_"+str(x) for x in np.arange(1,10)]
print("Columns to remove:",columns_to_remove)

Columns to remove: ['char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9']


In [200]:
#Remove the columns from the activity data
activity_df = activity_df[list(set(activity_df.columns) - set(columns_to_remove))]

In [201]:
activity_df.head()

,outcome,activity_category,char_10,activity_id,date,people_id
0,0,type 4,type 76,act2_1734928,2023-08-26,ppl_100
1,0,type 2,type 1,act2_2434093,2022-09-27,ppl_100
2,0,type 2,type 1,act2_3404049,2022-09-27,ppl_100
3,0,type 2,type 1,act2_3651215,2023-08-04,ppl_100
4,0,type 2,type 1,act2_4109017,2023-08-26,ppl_100


In [202]:
#Rename the 2 columns to avoid name clashes in merged data
activity_df = activity_df.rename(columns={"date":"activity_date","char_10":"activity_type"})

In [203]:
activity_df.head()

,outcome,activity_category,activity_type,activity_id,activity_date,people_id
0,0,type 4,type 76,act2_1734928,2023-08-26,ppl_100
1,0,type 2,type 1,act2_2434093,2022-09-27,ppl_100
2,0,type 2,type 1,act2_3404049,2022-09-27,ppl_100
3,0,type 2,type 1,act2_3651215,2023-08-04,ppl_100
4,0,type 2,type 1,act2_4109017,2023-08-26,ppl_100


In [204]:
#The mode of a set of data values is the value that appears most often.
activity_df["activity_type"].mode()

0    type 1
dtype: object

In [205]:
#Replace nulls in the activity_type column with the mode
activity_df["activity_type"] = activity_df["activity_type"].fillna(activity_df["activity_type"].mode()[0])

In [206]:
#Print the shape of the final activity dataset
print("Old Shape:","(2197291, 15)")
print("New Shape:",activity_df.shape)

Old Shape: (2197291, 15)
New Shape: (2197291, 6)


In [207]:
#Merge the 2 datasets on 'people_id' key
activity_df_new = activity_df.merge(people_df,on=["people_id"],how="inner")

In [208]:
activity_df_new.head()

,outcome,activity_category,activity_type,activity_id,activity_date,people_id,char_1,group_1,char_2,date,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,type 4,type 76,act2_1734928,2023-08-26,ppl_100,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
1,0,type 2,type 1,act2_2434093,2022-09-27,ppl_100,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
2,0,type 2,type 1,act2_3404049,2022-09-27,ppl_100,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
3,0,type 2,type 1,act2_3651215,2023-08-04,ppl_100,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36
4,0,type 2,type 1,act2_4109017,2023-08-26,ppl_100,type 2,group 17304,type 2,2021-06-29,...,False,True,True,False,False,True,True,True,False,36


In [209]:
print("Shape before merging:",activity_df.shape)
print("Shape after merging :",activity_df_new.shape)

Shape before merging: (2197291, 6)
Shape after merging : (2197291, 46)


##### The consistent number of rows and the increase in the number of columns helps us validate that the join operation worked as expected.

Let us now study the target (i.e., the variable we want to predict), named “outcome” in the dataset. 
We can check the distribution between potential vs. nonpotential customers.

In [210]:
print("Unique values for outcome:",activity_df["outcome"].unique())
print("\nPercentage of distribution for outcome-")
print(activity_df_new["outcome"].value_counts()/activity_df_new.shape[0])

Unique values for outcome: [0 1]

Percentage of distribution for outcome-
0    0.556046
1    0.443954
Name: outcome, dtype: float64


##### We can see that there is a good mix in the distribution of potentialcustomers, as around 45% are potential customers.

## 3) Data Engineering

In [211]:
#Checking the distinct datatypes in the dataset
print("Distinct DataTypes:",list(activity_df_new.dtypes.unique()))

Distinct DataTypes: [dtype('int64'), dtype('O'), dtype('bool')]


* We have numeric, categorical (Object), and Boolean features in the dataset. 
* Boolean in Python represents a True or False value; 
* We need to convert this into numeric (1 and 0) for the model to process the data.

In [214]:
#Create a temp dataset with the datatype of columns
datatype_ds = pd.DataFrame(activity_df_new.dtypes)
datatype_ds.columns = ["DataType"]

In [216]:
datatype_ds

,DataType
outcome,int64
activity_category,object
activity_type,object
activity_id,object
activity_date,object
people_id,object
char_1,object
group_1,object
char_2,object
date,object


In [217]:
#Create a list with names of all Boolean columns
boolean_columns = datatype_ds.index[temp["DataType"] == 'bool'].values
print("Boolean columns - \n",boolean_columns)

Boolean columns - 
 ['char_10' 'char_11' 'char_12' 'char_13' 'char_14' 'char_15' 'char_16'
 'char_17' 'char_18' 'char_19' 'char_20' 'char_21' 'char_22' 'char_23'
 'char_24' 'char_25' 'char_26' 'char_27' 'char_28' 'char_29' 'char_30'
 'char_31' 'char_32' 'char_33' 'char_34' 'char_35' 'char_36' 'char_37']


In [218]:
#Convert all boolean columns to Binary numeric values
for column in boolean_columns:
    activity_df_new[column] = np.where(activity_df_new[column] == True,1,0)

In [220]:
datatype_ds_new = pd.DataFrame(activity_df_new.dtypes)
datatype_ds_new.columns = ["DataType"]
datatype_ds_new

,DataType
outcome,int64
activity_category,object
activity_type,object
activity_id,object
activity_date,object
people_id,object
char_1,object
group_1,object
char_2,object
date,object


In [221]:
activity_df_new.head()

,outcome,activity_category,activity_type,activity_id,activity_date,people_id,char_1,group_1,char_2,date,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,type 4,type 76,act2_1734928,2023-08-26,ppl_100,type 2,group 17304,type 2,2021-06-29,...,0,1,1,0,0,1,1,1,0,36
1,0,type 2,type 1,act2_2434093,2022-09-27,ppl_100,type 2,group 17304,type 2,2021-06-29,...,0,1,1,0,0,1,1,1,0,36
2,0,type 2,type 1,act2_3404049,2022-09-27,ppl_100,type 2,group 17304,type 2,2021-06-29,...,0,1,1,0,0,1,1,1,0,36
3,0,type 2,type 1,act2_3651215,2023-08-04,ppl_100,type 2,group 17304,type 2,2021-06-29,...,0,1,1,0,0,1,1,1,0,36
4,0,type 2,type 1,act2_4109017,2023-08-26,ppl_100,type 2,group 17304,type 2,2021-06-29,...,0,1,1,0,0,1,1,1,0,36


In [222]:
print("\nDistinct DataTypes after processing:",activity_df_new.dtypes.unique())


Distinct DataTypes after processing: [dtype('int64') dtype('O') dtype('int32')]


In [223]:
#Extracting the object columns from the above dataframe
categorical_columns = datatype_ds_new.index[temp["DataType"] == 'O'].values

In [224]:
categorical_columns

array(['activity_category', 'activity_type', 'activity_id',
       'activity_date', 'people_id', 'char_1', 'group_1', 'char_2',
       'date', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8',
       'char_9'], dtype=object)

In [225]:
#Check the number of distinct values in each categorical column
for column in categorical_columns:
    print(column+" column has :",str(len(activity_df_new[column].unique()))+" distinct values")

activity_category column has : 7 distinct values
activity_type column has : 6515 distinct values
activity_id column has : 2197291 distinct values
activity_date column has : 411 distinct values
people_id column has : 151295 distinct values
char_1 column has : 2 distinct values
group_1 column has : 29899 distinct values
char_2 column has : 3 distinct values
date column has : 1196 distinct values
char_3 column has : 43 distinct values
char_4 column has : 25 distinct values
char_5 column has : 9 distinct values
char_6 column has : 7 distinct values
char_7 column has : 25 distinct values
char_8 column has : 8 distinct values
char_9 column has : 9 distinct values


The five highlighted columns as shown in the output have high numbers of distinct values. It would be difficult to convert them into a one-hot encoded form, as they will consume too much memory during processing. In case you have the luxury of surplus RAM, feel free to convert them to a one-hot encoded data form.

In [226]:
#Create date related features for 'date' in customer data
activity_df_new["date"] = pd.to_datetime(activity_df_new["date"])
activity_df_new["Year"] = activity_df_new["date"].dt.year
activity_df_new["Month"] = activity_df_new["date"].dt.month
activity_df_new["Quarter"] = activity_df_new["date"].dt.quarter
activity_df_new["Week"] = activity_df_new["date"].dt.week
activity_df_new["WeekDay"] = activity_df_new["date"].dt.weekday
activity_df_new["Day"] = activity_df_new["date"].dt.day

In [227]:
#Create date related features for 'date' in activity data
activity_df_new["activity_date"] = pd.to_datetime(activity_df_new["activity_date"])
activity_df_new["Activity_Year"] = activity_df_new["activity_date"].dt.year
activity_df_new["Activity_Month"] = activity_df_new["activity_date"].dt.month
activity_df_new["Activity_Quarter"] = activity_df_new["activity_date"].dt.quarter
activity_df_new["Activity_Week"] = activity_df_new["activity_date"].dt.week
activity_df_new["Activity_WeekDay"] = activity_df_new["activity_date"].dt.weekday
activity_df_new["Activity_Day"] = activity_df_new["activity_date"].dt.day

In [228]:
activity_df_new.head()

,outcome,activity_category,activity_type,activity_id,activity_date,people_id,char_1,group_1,char_2,date,...,Quarter,Week,WeekDay,Day,Activity_Year,Activity_Month,Activity_Quarter,Activity_Week,Activity_WeekDay,Activity_Day
0,0,type 4,type 76,act2_1734928,2023-08-26,ppl_100,type 2,group 17304,type 2,2021-06-29,...,2,26,1,29,2023,8,3,34,5,26
1,0,type 2,type 1,act2_2434093,2022-09-27,ppl_100,type 2,group 17304,type 2,2021-06-29,...,2,26,1,29,2022,9,3,39,1,27
2,0,type 2,type 1,act2_3404049,2022-09-27,ppl_100,type 2,group 17304,type 2,2021-06-29,...,2,26,1,29,2022,9,3,39,1,27
3,0,type 2,type 1,act2_3651215,2023-08-04,ppl_100,type 2,group 17304,type 2,2021-06-29,...,2,26,1,29,2023,8,3,31,4,4
4,0,type 2,type 1,act2_4109017,2023-08-26,ppl_100,type 2,group 17304,type 2,2021-06-29,...,2,26,1,29,2023,8,3,34,5,26


In [229]:
#Delete the original date columns
del(activity_df_new["date"])
del(activity_df_new["activity_date"])
print("Shape of data after create Date Features:",activity_df_new.shape)

Shape of data after create Date Features: (2197291, 56)


In [230]:
# activity_date and date deleted
activity_df_new.head()

,outcome,activity_category,activity_type,activity_id,people_id,char_1,group_1,char_2,char_3,char_4,...,Quarter,Week,WeekDay,Day,Activity_Year,Activity_Month,Activity_Quarter,Activity_Week,Activity_WeekDay,Activity_Day
0,0,type 4,type 76,act2_1734928,ppl_100,type 2,group 17304,type 2,type 5,type 5,...,2,26,1,29,2023,8,3,34,5,26
1,0,type 2,type 1,act2_2434093,ppl_100,type 2,group 17304,type 2,type 5,type 5,...,2,26,1,29,2022,9,3,39,1,27
2,0,type 2,type 1,act2_3404049,ppl_100,type 2,group 17304,type 2,type 5,type 5,...,2,26,1,29,2022,9,3,39,1,27
3,0,type 2,type 1,act2_3651215,ppl_100,type 2,group 17304,type 2,type 5,type 5,...,2,26,1,29,2023,8,3,31,4,4
4,0,type 2,type 1,act2_4109017,ppl_100,type 2,group 17304,type 2,type 5,type 5,...,2,26,1,29,2023,8,3,34,5,26


Let us now have a look at the remaining categorical columns, which have very high numbers of distinct values.

In [231]:
print(activity_df_new[["people_id","activity_type","activity_id","group_1"]].head())

  people_id activity_type   activity_id      group_1
0   ppl_100       type 76  act2_1734928  group 17304
1   ppl_100        type 1  act2_2434093  group 17304
2   ppl_100        type 1  act2_3404049  group 17304
3   ppl_100        type 1  act2_3651215  group 17304
4   ppl_100        type 1  act2_4109017  group 17304


* It seems that we can convert all of the preceding categorical columns into numeric by extracting the relevant numeric ID from each of them.
* Since each of these columns has values in the form of someText_ someNumber. 
* Rather than converting these categorical columns into a bloated one-hot encoded dataset, we can temporarily use them as numeric features. 
* However, if the performance of the model doesn’t reach our desired expectations after several experiments, we might have to revisit these features and try our best to incorporate them differently. But for now, we can consider them as numeric features.

In [232]:
#For people ID, we would need to extract values after '_'
activity_df_new.people_id = activity_df_new.people_id.apply(lambda x:x.split("_")[1])
activity_df_new.people_id = pd.to_numeric(activity_df_new.people_id)

In [233]:
#For activity ID also, we would need to extract values after '_'
activity_df_new.activity_id = activity_df_new.activity_id.apply(lambda x:x.split("_")[1])
activity_df_new.activity_id = pd.to_numeric(activity_df_new.activity_id)

In [234]:
#For group_1 , we would need to extract values after "
activity_df_new.group_1 = activity_df_new.group_1.apply(lambda x:x.split(" ")[1])
activity_df_new.group_1 = pd.to_numeric(activity_df_new.group_1)

In [235]:
#For activity_type , we would need to extract values after "
activity_df_new.activity_type = activity_df_new.activity_type.apply(lambda x:x.split(" ")[1])
activity_df_new.activity_type = pd.to_numeric(activity_df_new.activity_type)

In [236]:
#Double check the new values in the dataframe
print(activity_df_new[["people_id","activity_type","activity_id","group_1"]].head())

   people_id  activity_type  activity_id  group_1
0      100.0             76    1734928.0    17304
1      100.0              1    2434093.0    17304
2      100.0              1    3404049.0    17304
3      100.0              1    3651215.0    17304
4      100.0              1    4109017.0    17304


Next, let’s convert the remaining categorical columns, which have relatively low numbers of distinct values, to one-hot encoded form and render the final consolidated dataset.

In [237]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#Define a function that will intake the raw dataframe and the column name and return a one hot encoded DF
def create_ohe(df, col):
    le = LabelEncoder()
    a=le.fit_transform(activity_df_new[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col+ "_"+ str(i) for i in le.classes_]
    return(pd.DataFrame(ohe.fit_transform(a),columns = column_names))

In [238]:
datatype_ds_new

,DataType
outcome,int64
activity_category,object
activity_type,object
activity_id,object
activity_date,object
people_id,object
char_1,object
group_1,object
char_2,object
date,object


In [ ]:
#Since the above function converts the column, one at a time
#We create a loop to create the final dataset with all features

target = ["outcome"]

numeric_columns = list(set(temp.index[(temp.DataType =="float64") | (temp.DataType =="int64")].values) - set(target))

temp = df_new[numeric_columns]
for column in categorical_columns:
    temp_df = create_ohe(activity_df_new,column)
    temp = pd.concat([temp1,temp_df],axis=1)
    
print("\nShape of final df after onehot encoding:",temp.shape)

In [239]:
target = ["outcome"]

numeric_columns = list(set(datatype_ds_new.index[(temp.DataType =="float64") | (datatype_ds_new.DataType =="int64")].values) - set(target))
numeric_columns

['char_38']

In [240]:
temp = activity_df_new[numeric_columns]
temp

,char_38
0,36
1,36
2,36
3,36
4,36
5,36
6,76
7,76
8,99
9,99


In [ ]:
categorical_columns